In [ ]:
# formated Text
text2 =[
 "Hi, I'd like to place an order, please.",
 "Sure, here are several sentences where users are requesting food:",
 "Can I place an order for delivery, please?",
 "I'd like to order some food.",
 "Hello, can you help me place a food order?",
 "Hi, I'd like to get a pizza and a drink, please.",
 "Could you take my order for tonight’s dinner?",
 "I need to place an order for some snacks.",
 "Is it possible to order some food now?",
 "Hello, I’m looking to order a meal, could you assist?",
 "Can you take an order for a burger and fries?",
"Hi, I’d like to order something for lunch."]


In [1]:
import spacy

def extract_intent_with_spacy(text):
    """
    Uses spaCy to extract intent based on verb-noun patterns and entities
    """
    # Load the English model
    nlp = spacy.load("en_core_web_sm")
    
    # Process the text
    doc = nlp(text)
    
    # Extract the root verb which often indicates the action
    root_verb = None
    for token in doc:
        if token.dep_ == "ROOT" and token.pos_ == "VERB":
            root_verb = token.lemma_
    
    # Extract entities that might be targets of the action
    entities = [ent.text for ent in doc.ents]
    
    # Extract nouns that might be objects of the action
    nouns = [token.text for token in doc if token.pos_ == "NOUN"]
    
    # Simple rule-based intent extraction
    if root_verb == "book" and any(noun in ["flight", "ticket", "hotel", "reservation"] for noun in nouns):
        return "booking_intent"
    elif root_verb in ["search", "find", "look"] and any(noun in ["flight", "hotel", "restaurant"] for noun in nouns):
        return "search_intent"
    elif root_verb in ["cancel", "delete", "remove"]:
        return "cancellation_intent"
    elif root_verb in ["help", "support", "assist"]:
        return "help_intent"
    elif "thank" in text.lower() or "thanks" in text.lower():
        return "gratitude_intent"
    elif "bye" in text.lower() or "goodbye" in text.lower():
        return "goodbye_intent"
    else:
        return "unknown_intent"



In [1]:
text = "I book reservation hotels in Paris"

# print(extract_intent_with_spacy(text))

In [2]:
import os
os.getcwd()



'c:\\kiosks_\\all_trials'

# Intent Recognition using NLTK

In [9]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
# path =  os.makedirs("nltk_data", exist_ok= True)
# # Download necessary NLTK resources
# nltk.download('punkt')
# nltk.download('wordnet')


def extract_intent_with_nltk(text):
    """
    Uses NLTK for keyword-based intent recognition
    """
    # Initialize lemmatizer

    nltk.data.path.append('nltk_data')
    lemmatizer = WordNetLemmatizer()
    
    # Convert to lowercase and tokenize
    tokens = word_tokenize(text.lower())
    
    # Lemmatize tokens
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]
    
    # Define intent keywords
    intent_keywords = {
        "booking_intent": ["book", "reserve", "schedule", "appointment", "reservation"],
        "odering_intent": ["order", "want", "place for","get", "get me", "add"],
        "weather_intent": ["weather", "temperature", "forecast", "rain", "sunny", "cloudy"],
        "navigation_intent": ["navigate", "direction", "map", "route", "go to", "take me to"],
        "farewell_intent": ["goodbye", "bye", "see you", "farewell", "later"],
        "gratitude_intent": ["thank", "thanks", "appreciate", "grateful"]
    }
    
    # Count matches for each intent
    intent_matches = {}
    for intent, keywords in intent_keywords.items():
        matches = sum(1 for token in lemmatized_tokens if token in keywords)
        intent_matches[intent] = matches
    
    # Return the intent with the most keyword matches
    if max(intent_matches.values(), default=0) > 0:
        return max(intent_matches.items(), key=lambda x: x[1])[0]
    else:
        return "unknown_intent"


In [10]:
# "odering_intent": ["order", "want", "place for", "get me", "add"],
# 
text =[
    'hey add two coffees for me',
    'hey just get me couple of cofee',
    'hey can you order me biryani',
    'from moring i want to eat biryani',
    'hey just place order for me'
]

for num,i in enumerate(text):
    print(f'{num+1}.{i} => {extract_intent_with_nltk(i)}')

1.hey add two coffees for me => odering_intent
2.hey just get me couple of cofee => odering_intent
3.hey can you order me biryani => odering_intent
4.from moring i want to eat biryani => odering_intent
5.hey just place order for me => odering_intent


In [6]:
intent_keywords = {
        "booking_intent": ["book", "reserve", "schedule", "appointment", "reservation"],
        "odering_intent": ["order", "want", "place for","get", "get me", "add"],
        "weather_intent": ["weather", "temperature", "forecast", "rain", "sunny", "cloudy"],
        "navigation_intent": ["navigate", "direction", "map", "route", "go to", "take me to"],
        "farewell_intent": ["goodbye", "bye", "see you", "farewell", "later"],
        "gratitude_intent": ["thank", "thanks", "appreciate", "grateful"]
    }

for intent, keywords in intent_keywords.items():
    print(f'Intent: {intent}')
    print(f'keywords: {keywords}')



Intent: booking_intent
keywords: ['book', 'reserve', 'schedule', 'appointment', 'reservation']
Intent: odering_intent
keywords: ['order', 'want', 'place for', 'get', 'get me', 'add']
Intent: weather_intent
keywords: ['weather', 'temperature', 'forecast', 'rain', 'sunny', 'cloudy']
Intent: navigation_intent
keywords: ['navigate', 'direction', 'map', 'route', 'go to', 'take me to']
Intent: farewell_intent
keywords: ['goodbye', 'bye', 'see you', 'farewell', 'later']
Intent: gratitude_intent
keywords: ['thank', 'thanks', 'appreciate', 'grateful']


# Using bart-large-mnli zero shot classifiation

In [29]:
from transformers import pipeline

def extract_intent_with_transformers(text):
    """
    Uses Hugging Face transformers for zero-shot classification of intents
    """
    # Initialize zero-shot classification pipeline
    classifier = pipeline("zero-shot-classification",
                      model="facebook/bart-large-mnli")

    
    # Define possible intents
    candidate_intents = [
        "booking", "reservation", "search", "finding information"
    ]
    
    # Classify the text
    result = classifier(text, candidate_intents)
    
    # Return the highest scoring intent
    top_intent = result['labels'][0]
    
    # Map the result to our intent format
    intent_mapping = {
        "booking": "booking_intent",
        "reservation": "booking_intent",
        "search": "search_intent",
        "finding information": "search_intent",
        
    }
    
    return intent_mapping.get(top_intent, "unknown_intent")


In [39]:
ans = []
for num, i in enumerate(text2):
    try:
        a = extract_intent_with_transformers(i)
        ans.append(f'{num+1}.{i} => {a}')
    except Exception as e:
        print(f"Error processing: {i}, Error: {e}")

print(ans)

Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu


["1.Hi, I'd like to place an order, please. => booking_intent", '2.Sure, here are several sentences where users are requesting food: => booking_intent', '3.Can I place an order for delivery, please? => booking_intent', "4.I'd like to order some food. => booking_intent", '5.Hello, can you help me place a food order? => booking_intent', "6.Hi, I'd like to get a pizza and a drink, please. => booking_intent", '7.Could you take my order for tonight’s dinner? => booking_intent', '8.I need to place an order for some snacks. => booking_intent', '9.Is it possible to order some food now? => search_intent', '10.Hello, I’m looking to order a meal, could you assist? => booking_intent', '11.Can you take an order for a burger and fries? => booking_intent', '12.Hi, I’d like to order something for lunch. => booking_intent']


In [33]:
ans

['5.hey just place order for me => booking_intent']